In [ ]:
# @title 🚀 全面升級數據整合平台 (v17.1) - 啟動器
#@markdown ---
#@markdown ### **核心架構：模組化、高穩定性、GitHub 直連**
#@markdown 歡迎使用數據整合平台 v17.1！此版本基於強化的模組化工程構建，提供前所未有的執行穩定性與更清晰的程式碼結構。
#@markdown - **智慧型程式碼更新**：啟動器會自動從您指定的 GitHub 儲存庫拉取最新程式碼。
#@markdown - **一站式參數配置**：所有可調整參數均集中於下方，方便您快速完成設定。
#@markdown - ✨ **新增功能**：執行完畢後，將在下方提供「一鍵複製全部日誌」按鈕，方便您在行動裝置上操作。
#@markdown ---
#@markdown ### ⚙️ **執行核心設定**
#@markdown ---
#@markdown #### **(A) 系統與程式碼來源**
#@markdown 指定數據平台的核心程式碼來源。除非您是開發者或使用自訂版本，否則請保留預設值。
repo_url = "https://github.com/hsp1234-web/taifexd-date.git" #@param {type:"string"}
#@markdown ---
#@markdown #### **(B) 執行模式選擇**
#@markdown ✅ **Google Drive 整合模式**：勾選此項，您的專案資料夾與產出將同步並永久保存於您的 Google Drive。
#@markdown ❌ **本地臨時模式**：取消勾選，所有檔案將儲存於此 Colab 的臨時空間，適合快速測試，但關閉分頁後資料會遺失。
enable_google_drive_integration = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **(C) 主要路徑與檔案名稱**
#@markdown 設定您在 Google Drive 中的專案根目錄，以及輸出的資料庫與日誌檔名。
project_folder = "MyTaifexDataProject" #@param {type:"string"}
database_name = "processed_data.duckdb" #@param {type:"string"}
log_name = "pipeline.log" #@param {type:"string"}
#@markdown --- 
#@markdown #### **(D) 特定檔案處理**
#@markdown 若要指定處理單一或多個ZIP檔，請在此輸入檔名，並用逗號 (`,`) 分隔。留空則代表處理輸入目錄中的全部 ZIP 檔案。
target_zip_files = "" #@param {type:"string"}
#@markdown ---
#@markdown #### **(E) 開發者選項 (Developer)**
#@markdown ⚠️ **除錯模式**：僅在需要深入分析問題時開啟。將啟用詳細日誌與硬體效能監控，日誌檔案會變得非常大。
debug_mode = False #@param {type:"boolean"}

# ======================================================================
#         穩健執行區 (請勿修改此線以下內容)
# ======================================================================
import os
import sys
import shutil
import io
import contextlib
import base64
from google.colab import drive
from IPython.display import display, HTML

# --- 日誌捕捉與即時顯示的核心類別 ---
class OutputTee(io.StringIO):
    """一個特殊的類別，可以將輸出同時寫入原始串流和內存緩衝區。"""
    def __init__(self, original_stream):
        super().__init__()
        self._original_stream = original_stream

    def write(self, s):
        self._original_stream.write(s) # 即時顯示
        super().write(s)               # 寫入緩衝區

    def flush(self):
        self._original_stream.flush()
        super().flush()

# --- 主要執行邏輯 ---
log_capture = OutputTee(sys.stdout)
final_status = "✅ 正常結束"

with contextlib.redirect_stdout(log_capture), contextlib.redirect_stderr(log_capture):
    try:
        print("--- 參數動態顯示 (執行時更新) ---")
        print(f"程式碼來源: {repo_url}")
        print(f"Google Drive 整合模式: {'啟用' if enable_google_drive_integration else '停用 (本地測試模式)'}")
        print(f"專案資料夾: {project_folder}")
        print(f"輸出資料庫: {database_name}")
        print(f"輸出日誌檔: {log_name}")
        print(f"指定ZIP檔: {'全部' if not target_zip_files else target_zip_files}")
        print("--------------------\n")

        # 步驟 1: 掛載 Google Drive (如果啟用)
        if enable_google_drive_integration:
            try:
                print("⏳ 正在請求掛載 Google Drive...")
                drive.mount('/content/drive', force_remount=True)
                print("✅ Google Drive 掛載成功！")
            except Exception as e:
                print(f"❌ Google Drive 掛載失敗: {e}")
                raise SystemExit("Google Drive 掛載失敗，請檢查授權。")
        else:
            print("ℹ️ Google Drive 整合已停用。所有操作將在本地臨時環境進行。")
        print("\n")

        # 步驟 2: 下載/更新專案程式碼
        print("⏳ 正在準備專案程式碼...")
        os.chdir('/content')
        repo_name = 'taifexd-date'
        repo_path = os.path.join('/content', repo_name)

        if os.path.exists(repo_path):
            print(f"  - 偵測到舊的 '{repo_path}' 目錄，正在刪除以確保同步最新版本...")
            shutil.rmtree(repo_path)

        print(f"  - 執行 git clone，開始從 {repo_url} 下載專案...")
        get_ipython().system(f"git clone -q {repo_url} {repo_name}")

        if not os.path.exists(repo_path):
            raise SystemExit(f"Git Clone 失敗! 請檢查 URL: {repo_url}")

        os.chdir(repo_path)
        print(f"✅ 專案程式碼準備完成！目前工作目錄: {os.getcwd()}\n")

        # 步驟 3: 同步鎖定檔並安裝依賴
        print("⏳ 正在同步 lock 檔案並安裝依賴 (此過程可能需要數分鐘，請稍候)...")
        get_ipython().system("pip install poetry -q")
        print("  - 執行 poetry lock 來產生與 pyproject.toml 同步的鎖定檔案...")
        get_ipython().system("poetry lock")
        print("  - 執行 poetry install 來安裝所有必要的套件...")
        get_ipython().system("poetry install --no-root --without dev")
        print("✅ 所有精確版本的 Python 套件均已安裝完成！\n")

        # 步驟 4: 建構並執行主程式
        print("🚀 即將啟動數據整合平台主程式...")
        command_parts = [
            "poetry run python main.py",
            f"--project-folder-name='{project_folder}'",
            f"--database-name='{database_name}'",
            f"--log-name='{log_name}'",
            f"--zip-files='{target_zip_files}'"
        ]
        if not enable_google_drive_integration:
            command_parts.append("--no-gdrive")

        if debug_mode:
            command_parts.append("--debug")

        command = " ".join(command_parts)

        print(f"  - 將執行的最終指令:\n    {command}")
        print("-" * 20 + " 程式開始執行 " + "-" * 20)

        # 執行主命令並即時輸出
        process = get_ipython().getoutput(command, split=False)
        print(process)

        print("-" * 20 + " 程式執行完畢 " + "-" * 20)

    except SystemExit as e:
        final_status = f"ℹ️ 執行已中止：{e}"
        print(f"\n{final_status}")
    except Exception as e:
        final_status = f"❌ 發生未預期的錯誤：{e}"
        print(f"\n{final_status}")

# --- 產出最終結果與複製按鈕 ---
full_log_content = log_capture.getvalue()
encoded_logs = base64.b64encode(full_log_content.encode('utf-8')).decode('utf-8')

html_output = f"""\
<div style="border: 2px solid #4CAF50; padding: 10px; margin-top: 20px; border-radius: 8px; background-color: #f0fff0;">\
    <h3 style='color: #2E7D32; margin-top:0;'>📋 執行結果摘要</h3>\
    <p><strong>最終狀態：</strong>{final_status}</p>\
    <button \
        onclick="copyLogsToClipboard()" \
        style="background-color: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer; font-size: 14px;"\
    >\
        一鍵複製上方完整日誌\
    </button>\
    <p id="copy-status" style='font-size: 12px; color: #555; margin-top: 8px;'>點擊按鈕後，所有執行過程的詳細日誌將複製到您的剪貼簿。</p>\
</div>\
<textarea id="hidden-logs" style="opacity: 0; position: absolute; left: -9999px;"></textarea>\
<script>\
  (function() {{ \
    try {{ \
        document.getElementById('hidden-logs').value = atob('{encoded_logs}');\
    }} catch (e) {{ \
        console.error('Base64 解碼失敗:', e);\
        document.getElementById('hidden-logs').value = '日誌內容解碼失敗，可能包含特殊字元。';\
    }} \
  }})();\
  \
  function copyLogsToClipboard() {{ \
    const textarea = document.getElementById('hidden-logs');\
    const status_p = document.getElementById('copy-status');\
    textarea.select();\
    textarea.setSelectionRange(0, 999999);\
    try {{ \
      const successful = document.execCommand('copy');\
      if (successful) {{ \
        status_p.textContent = '✅ 已成功複製到剪貼簿！';\
        status_p.style.color = 'green';\
      }} else {{ \
        status_p.textContent = '❌ 複製失敗。您的瀏覽器可能不支援此操作。';\
        status_p.style.color = 'red';\
      }} \
    }} catch (err) {{ \
      status_p.textContent = '❌ 複製時發生錯誤，請手動複製。';\
      status_p.style.color = 'red';\
      console.error('複製日誌失敗: ', err);\
    }} \
    setTimeout(() => {{ \
        status_p.textContent = '點擊按鈕後，所有執行過程的詳細日誌將複製到您的剪貼簿。';\
        status_p.style.color = '#555';\
    }}, 3000);\
  }} \
</script>\
"""

display(HTML(html_output))
